In [1]:
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.ml.feature import NGram, Tokenizer
import pandas as pd

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
companies = spark.read.csv(
            "companies.csv", 
            header=True, inferSchema=True)

In [4]:
jobs = spark.read.csv(
            "alldatas.csv", 
            header=True, inferSchema=True)

In [183]:
companies = companies.withColumnRenamed('description', 'desc')


In [182]:
# df1.join(df2, df1.id == df2.id).select('df1.*')
# companies = companies.select(col("description").alias("desc"))
aaa = companies.join(jobs, companies.industry == jobs.description)



In [185]:
tokens = Tokenizer(inputCol='desc', outputCol='tokens')
nn = NGram(n=2, inputCol='tokens', outputCol='ngrams')

In [186]:
b = tokens.transform(companies)
a = nn.transform(b)
final = a.select(['tokens', 'ngrams']).show(3)


+--------------------+--------------------+
|              tokens|              ngrams|
+--------------------+--------------------+
|[agilent, technol...|[agilent technolo...|
|[alcoa, corp, is,...|[alcoa corp, corp...|
|[altaba, inc, is,...|[altaba inc, inc ...|
+--------------------+--------------------+
only showing top 3 rows



In [106]:
def ngrram(dataframe, column, x):
    tokens = Tokenizer(inputCol=column, outputCol='tokens')
    nn = NGram(n=x, inputCol='tokens', outputCol='ngrams')
    b = tokens.transform(dataframe)
    a = nn.transform(b)
    final = a.select(['tokens', 'ngrams']).show(4)
    return final

In [187]:
ngrram(companies, 'desc', 2)


+--------------------+--------------------+
|              tokens|              ngrams|
+--------------------+--------------------+
|[agilent, technol...|[agilent technolo...|
|[alcoa, corp, is,...|[alcoa corp, corp...|
|[altaba, inc, is,...|[altaba inc, inc ...|
|[aac, holdings, i...|[aac holdings, ho...|
+--------------------+--------------------+
only showing top 4 rows



In [188]:
loc = a.select('location', F.split(a['location'], ',')[0].alias('city')).show(10)

# a.columns
# a.select('city').show()

+------------------+--------+
|          location|    city|
+------------------+--------+
|Atlanta, GA 30301 | Atlanta|
|       Atlanta, GA| Atlanta|
|       Atlanta, GA| Atlanta|
|Atlanta, GA 30303 | Atlanta|
|       Atlanta, GA| Atlanta|
|          AADR.png|AADR.png|
|Atlanta, GA 30318 | Atlanta|
|Atlanta, GA 30318 | Atlanta|
|       Atlanta, GA| Atlanta|
|       Atlanta, GA| Atlanta|
+------------------+--------+
only showing top 10 rows



In [189]:
ff = a.filter(a.desc.isNotNull()).select(['ngrams', 'location']).select(F.explode('ngrams').alias('ngrams'), 'location', F.split(a['location'], ',')[0].alias('city')).groupBy(['ngrams', 'city']).count()

In [190]:
ff.show(5)

+--------------------+-------+-----+
|              ngrams|   city|count|
+--------------------+-------+-----+
|diagnostics labor...|Atlanta|    1|
|capital appreciation|Atlanta|    2|
|          that helps|Atlanta|    1|
|       and solutions|Atlanta|    4|
|    services include|Atlanta|    1|
+--------------------+-------+-----+
only showing top 5 rows



In [192]:
gg = a.filter(a.desc.isNotNull()).select(['ngrams', 'industry']).select(F.explode('ngrams').alias('ngrams'), 'industry', F.split(a['industry'], ',')[0].alias('industries')).groupBy(['ngrams', 'industry']).count()

In [193]:
gg.show()

+--------------------+--------------------+-----+
|              ngrams|            industry|count|
+--------------------+--------------------+-----+
|    washington, d.c.|            Airlines|    1|
|    provider serving|Retail - Apparel ...|    1|
|plasma-based prod...|       Biotechnology|    1|
|   asbury automotive|               Autos|    1|
|interception, geo...|Communication Equ...|    1|
|         markets. it|Engineering & Con...|    1|
|             for the|               REITs|    4|
|               and a|Utilities - Regul...|    1|
|            the usa,|           Insurance|    1|
|       canadian gold|     Metals & Mining|    1|
|      namely aethlon|   Computer Hardware|    1|
|         provider of| Industrial Products|   11|
|      utilities, and|     Credit Services|    2|
|      for automotive|Homebuilding & Co...|    1|
| wireless networking|Communication Equ...|    1|
|           arthur j.| Brokers & Exchanges|    1|
|         and branded|  Drug Manufacturers|    3|
